<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Attention_attribution_cosine_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook explores the relation between the model's attributions and attentions for a given example. Historically, we found that attentions are not a feasible method of explanation whereas attributions are, but attributions are also not part of a model's traditional outputs. Therefore it may be interesting to see if we can find anything with attentions by comparing them to a feasible and plausible method of explanation.

In [332]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import dependencies

In [333]:
pip install transformers --quiet

In [334]:
pip install captum --quiet

In [335]:
pip install datasets --quiet

In [336]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [337]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [338]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

In [339]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
#model_path = 'danielhou13/longformer-finetuned-new-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Create functions that give us the input ids and the position ids for the text we want to examine

In [340]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [341]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [342]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

Import dataset and take a few examples from it for testing purposes

Here we import the papers dataset

In [343]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Using custom data configuration danielhou13--cogs402dataset-144b958ac1a53abb
Reusing dataset parquet (/root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-144b958ac1a53abb/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/2 [00:00<?, ?it/s]

Here we import the news dataset

In [344]:
# cogs402_ds2 = load_dataset('hyperpartisan_news_detection', 'bypublisher')['validation']
# val_size = 5000
# val_indices = np.random.randint(0, len(cogs402_ds2), val_size)
# val_ds = cogs402_ds2.select(val_indices)
# labels2 = map(int, val_ds['hyperpartisan'])
# labels2 = list(labels2)
# val_ds = val_ds.add_column("labels", labels2)

In [345]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

Perform Layer Integrated Gradients using the longformer's embeddings

In [346]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    return attributions

In [347]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

This function will let us get the example and the baseline inputs in order to perform integrated gradients, and add the attributions to our visualization tool. Additionally, we will add the attributions and tokens for each example into an array so we can use them when we want to further example the attributions scores for each example

In [348]:
all_attributions = {}
all_tokens = {}

In [487]:
all_attributions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict.pt')

In [776]:
example = 605
text = cogs402_ds['text'][example]
label = cogs402_ds['labels'][example]

input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens_curr = tokenizer.convert_ids_to_tokens(indices)

all_tokens[str(example)] = all_tokens_curr

In [777]:
# attributions, delta = lig.attribute(inputs=input_ids,
#                                   baselines=ref_input_ids,
#                                   return_convergence_delta=True,
#                                   additional_forward_args=(position_ids, attention_mask),
#                                   target=1,
#                                   n_steps=1500,
#                                   internal_batch_size = 2)

# attributions_sum = summarize_attributions(attributions)

# all_attributions[str(example)] = attributions_sum.detach().cpu().numpy()

In [778]:
# torch.save(all_attributions, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict.pt')

In [779]:
# all_attributions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict.pt')

In [780]:
print(all_attributions)

{'605': array([ 0.00000000e+00, -3.40173878e-06, -1.12807776e-06, ...,
        4.23677990e-06,  7.29073352e-06,  0.00000000e+00]), '976': array([ 0.00000000e+00, -1.84040401e-04,  1.46363292e-05, ...,
       -2.48273481e-06,  1.11380563e-04,  0.00000000e+00]), '148': array([ 0.        ,  0.00225466,  0.00335924, ...,  0.00014387,
       -0.000275  ,  0.        ]), '891': array([ 0.00000000e+00, -1.05098504e-03, -2.83239306e-03, ...,
        1.52405500e-06,  9.14739375e-05,  0.00000000e+00])}


We then get the attentions and global attentions so we can compare with the attributions

In [781]:
output = model(input_ids.cuda(), attention_mask=attention_mask.cuda(), labels=torch.tensor(label).cuda(), output_attentions = True)
batch_attn = output[-2]
output_attentions = torch.stack(batch_attn).cpu()
global_attention = output[-1]
output_global_attentions = torch.stack(global_attention).cpu()
print("output_attention.shape", output_attentions.shape)
print("gl_output_attention.shape", output_global_attentions.shape)

output_attention.shape torch.Size([12, 1, 12, 2048, 514])
gl_output_attention.shape torch.Size([12, 1, 12, 2048, 1])


Since the longformer has a unique attention matrix shape, we convert it into the required sequence length x sequence length matrix

In [782]:
def create_head_matrix(output_attentions, global_attentions):
    new_attention_matrix = torch.zeros((output_attentions.shape[0], 
                                      output_attentions.shape[0]))
    for i in range(output_attentions.shape[0]):
        test_non_zeroes = torch.nonzero(output_attentions[i]).squeeze()
        test2 = output_attentions[i][test_non_zeroes[1:]]
        new_attention_matrix_indices = test_non_zeroes[1:]-257 + i
        new_attention_matrix[i][new_attention_matrix_indices] = test2
        new_attention_matrix[i][0] = output_attentions[i][0]
        new_attention_matrix[0] = global_attentions.squeeze()[:output_attentions.shape[0]]
    return new_attention_matrix


def attentions_all_heads(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = create_head_matrix(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_batches(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = attentions_all_heads(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_layers(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = all_batches(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

In [783]:
converted_mat = all_layers(output_attentions, output_global_attentions).detach().cpu().numpy()
print(converted_mat.shape)

(12, 1, 12, 2048, 2048)


We scale the attention matrix by head importance

In [784]:
head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/papers/pretrained/head_importance.pt")
# head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/news/head_importance.pt")

In [785]:
def scale_by_importance(attention_matrix, head_importance):
  new_matrix = np.zeros_like(attention_matrix)
  for i in range(attention_matrix.shape[0]):
    head_importance_layer = head_importance[i]
    for j in range(attention_matrix.shape[1]):
      new_matrix[i,j] = attention_matrix[i,j] * np.expand_dims(head_importance_layer, axis=(1,2))
  return new_matrix

In [786]:
converted_mat_importance = scale_by_importance(converted_mat, head_importance)

We get the attentions for each token. The shape of the attention matrix is layer x batch x head x seq_len x seq_len.

In [787]:
attention_matrix_importance = converted_mat_importance.sum(axis=3)
print(attention_matrix_importance.shape)

(12, 1, 12, 2048)


Sum the attentions for the last layer and over all layers

In [788]:
attention_final_layer = attention_matrix_importance[11].squeeze().sum(axis=0)
attention_all_layer = attention_matrix_importance.squeeze().sum(axis=1)
attention_all_layer = attention_all_layer.sum(axis=0)
print(attention_all_layer.shape)

(2048,)


Grab the attributions we stored

In [789]:
exam_attrib = all_attributions[str(example)]

Since we have the attributions and the attentions, we want to see how the largest attributions (in terms of magnitude) compares to the highest attentions.

Cosine similarity using the raw attributions and attentions

In [790]:
from numpy.linalg import norm
cosine_raw = np.dot(exam_attrib, attention_final_layer) / (norm(exam_attrib)*norm(attention_final_layer))
print("Layer 12 Cosine Similarity raw attrib:\n", cosine_raw)
cosine_all_raw = np.dot(exam_attrib, attention_all_layer) / (norm(exam_attrib)*norm(attention_all_layer))
print("Layer 12 Cosine Similarity raw attrib:\n", cosine_all_raw)

Layer 12 Cosine Similarity raw attrib:
 -0.16550712124895883
Layer 12 Cosine Similarity raw attrib:
 -0.21994437908930428


The attributions and the attentions have different ranges. The attributions could range from -1 to 1 whereas the attentions range from 0 to 1. However, negative attributions would not necessarily mean that they have the lowest attention, rather they might have really high attention as they are more likely to help the model predict the negative class, and might be something the attentions picked up as a feature.

In [791]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [792]:
attention_final_layer2 = normalize(attention_final_layer)
attention_all_layer2 = normalize(attention_all_layer)

In [793]:
exam_attrib2 = np.abs(exam_attrib)
exam_attrib2 = normalize(exam_attrib2)

In [794]:
print(exam_attrib2)

[0.         0.0561608  0.01862393 ... 0.06994687 0.12036593 0.        ]


Calculate cosine simularity using normalized attentions and attributions

In [795]:
cosine = np.dot(exam_attrib2, attention_final_layer2) / (norm(exam_attrib2)*norm(attention_final_layer2))
print("Layer 12 Cosine Similarity:\n", cosine)
cosine2 = np.dot(exam_attrib2, attention_all_layer2) / (norm(exam_attrib2)*norm(attention_all_layer2))
print("All layer Cosine Similarity:\n", cosine2)

Layer 12 Cosine Similarity:
 0.2993751970093294
All layer Cosine Similarity:
 0.24973547539622828


Cosine similarity while setting all the attention and attribution values below the median to 0

In [796]:
exam_attrib3 = np.abs(exam_attrib)
exam_attrib3 = normalize(exam_attrib3)
median_exam = np.percentile(exam_attrib3, 50)
exam_attrib3[exam_attrib3 < median_exam] = 0

In [797]:
attention_final_layer3 = np.copy(attention_final_layer)
attention_final_layer3 = normalize(attention_final_layer3)
median_12 = np.percentile(attention_final_layer3, 50)
attention_final_layer3[attention_final_layer3 < median_12] = 0

attention_all_layer3 = np.copy(attention_all_layer) 
attention_all_layer3 = normalize(attention_all_layer3)
median_all = np.percentile(attention_all_layer3, 50)
attention_all_layer3[attention_all_layer3 < median_all] = 0

In [798]:
cosine_med = np.dot(exam_attrib3, attention_final_layer3) / (norm(exam_attrib3)*norm(attention_final_layer3))
print("Layer 12 Cosine Similarity med:\n", cosine_med)
cosine_med2 = np.dot(exam_attrib3, attention_all_layer3) / (norm(exam_attrib3)*norm(attention_all_layer3))
print("All layer Cosine Similarity med:\n", cosine_med2)

Layer 12 Cosine Similarity med:
 0.28822077989088346
All layer Cosine Similarity med:
 0.2331588710720503


Cosine similarity while setting all the attention and attribution values below the mean to 0

In [799]:
exam_attrib4 = np.abs(exam_attrib)
exam_attrib4 = normalize(exam_attrib4)
mean_exam = np.mean(exam_attrib4)
exam_attrib4[exam_attrib4 < mean_exam] = 0

In [800]:
attention_final_layer4 = np.copy(attention_final_layer)
attention_final_layer4 = normalize(attention_final_layer4)
mean_12 = np.mean(attention_final_layer4)
attention_final_layer4[attention_final_layer4 < mean_12] = 0

attention_all_layer4 = np.copy(attention_all_layer) 
attention_all_layer4 = normalize(attention_all_layer4)
mean_all = np.mean(attention_all_layer4)
attention_all_layer4[attention_all_layer4 < mean_all] = 0

In [801]:
cosine_mean = np.dot(exam_attrib4, attention_final_layer4) / (norm(exam_attrib4)*norm(attention_final_layer4))
print("Layer 12 Cosine Similarity mean:\n", cosine_mean)
cosine_mean2 = np.dot(exam_attrib4, attention_all_layer4) / (norm(exam_attrib4)*norm(attention_all_layer4))
print("All layer Cosine Similarity mean:\n", cosine_mean2)

Layer 12 Cosine Similarity mean:
 0.27112812192382313
All layer Cosine Similarity mean:
 0.21981108786328662


Cosine Similarity using the ranks of each token

In [802]:
exam_attrib_rank = np.abs(exam_attrib)
order_attrib = exam_attrib_rank.argsort()
print(order_attrib)
ranks_attrib = order_attrib.argsort()
print(ranks_attrib)

[   0 2047  263 ... 1024  102  989]
[   0 1899 1247 ... 1941 2002    1]


In [803]:
attention_final_layer_rank = np.copy(attention_final_layer)
order = attention_final_layer_rank.argsort()
ranks = order.argsort()

attention_all_layer_rank = np.copy(attention_all_layer)
order2 = attention_all_layer_rank.argsort()
ranks2 = order2.argsort()

In [804]:
cosine_rank = np.dot(ranks_attrib, ranks) / (norm(ranks_attrib)*norm(ranks))
print("Layer 12 Cosine Similarity rank:\n", cosine_rank)
cosine_rank2 = np.dot(ranks_attrib, ranks2) / (norm(ranks_attrib)*norm(ranks2))
print("All layer Cosine Similarity rank:\n", cosine_rank2)

Layer 12 Cosine Similarity rank:
 0.8020630713097967
All layer Cosine Similarity rank:
 0.7845054106949373


The cosine similarity using only the last layer of attentions

In [805]:
d = {'example': [example], 'similarity normalized': [cosine], 'similarity raw': [cosine_raw], 'sim_norm w/ median threshold':[cosine_med], 'sim_norm w/ mean threshold':[cosine_mean], "sim w/ ranks":cosine_rank}
df = pd.DataFrame(data=d)
df

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold,sim w/ ranks
0,605,0.299375,-0.165507,0.288221,0.271128,0.802063


The cosine similarity using all layers

In [806]:
d2 = {'example': [example], 'similarity normalized': [cosine2], 'similarity raw': [cosine_all_raw], 'sim_norm w/ median threshold':[cosine_med2], 'sim_norm w/ mean threshold':[cosine_mean2], "sim w/ ranks":cosine_rank2}
df2 = pd.DataFrame(data=d2)
df2

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold,sim w/ ranks
0,605,0.249735,-0.219944,0.233159,0.219811,0.784505


In [807]:
df_layer12 = pd.read_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/cos_sim_layer12.csv")
df_all = pd.read_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/cos_sim_all.csv")

In [808]:
df_layer12

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold,sim w/ ranks
0,148,0.147101,-0.020260,0.142670,0.123295,0.826202
1,605,0.299375,-0.165507,0.288221,0.271128,0.802063
2,891,0.175846,0.022566,0.173801,0.157258,0.842796
3,976,0.141923,0.082403,0.137433,0.119868,0.823112


In [809]:
df_all

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold,sim w/ ranks
0,148,0.140745,0.016048,0.134218,0.123968,0.805108
1,605,0.249735,-0.219944,0.233159,0.219811,0.784505
2,891,0.104379,-0.012725,0.100741,0.089762,0.799092
3,976,0.115270,0.096755,0.109309,0.097665,0.819613


Append the new row into the dataframe.

Comment out if revisiting a dataframe.

In [810]:
# df_layer12 = pd.concat([df, df_layer12], axis=0)
# df_all = pd.concat([df2, df_all], axis=0)

In [811]:
df_layer12

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold,sim w/ ranks
0,148,0.147101,-0.020260,0.142670,0.123295,0.826202
1,605,0.299375,-0.165507,0.288221,0.271128,0.802063
2,891,0.175846,0.022566,0.173801,0.157258,0.842796
3,976,0.141923,0.082403,0.137433,0.119868,0.823112


In [812]:
df_all

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold,sim w/ ranks
0,148,0.140745,0.016048,0.134218,0.123968,0.805108
1,605,0.249735,-0.219944,0.233159,0.219811,0.784505
2,891,0.104379,-0.012725,0.100741,0.089762,0.799092
3,976,0.115270,0.096755,0.109309,0.097665,0.819613


In [813]:
df_layer12 = df_layer12.sort_values(by=['example'])
df_all = df_all.sort_values(by=['example'])

Save the dataframe

In [814]:
# df_layer12.to_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/cos_sim_layer12.csv", index=False)
# df_all.to_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/cos_sim_all.csv", index=False)

We know from the cosine similarities that it does not seem like the attribtions and the attentions are very similar; however, we can find out if there are similarities in the tokens in the highest percentiles.


In [815]:
attention_final_layer5 = np.copy(attention_final_layer)
attention_final_layer5 = normalize(attention_final_layer5)

attention_all_layer5 = np.copy(attention_all_layer) 
attention_all_layer5 = normalize(attention_all_layer5)

exam_attrib5 = np.abs(exam_attrib)
exam_attrib5 = normalize(exam_attrib5)
print(exam_attrib5)

[0.         0.0561608  0.01862393 ... 0.06994687 0.12036593 0.        ]


In [816]:
top_final = np.percentile(attention_final_layer5, 95)
top_all = np.percentile(attention_final_layer5, 95)
top_attrib = np.percentile(exam_attrib5, 95)
print(top_attrib)

0.07131846942409473


In [817]:
attention_final_layer5[attention_final_layer5<top_final] = 0
attention_all_layer5[attention_all_layer5<top_all] = 0
exam_attrib5[exam_attrib5<top_attrib] = 0

In [818]:
print(exam_attrib5)

[0.         0.         0.         ... 0.         0.12036593 0.        ]


In [819]:
cosine_thresh = np.dot(exam_attrib5, attention_final_layer5) / (norm(exam_attrib5)*norm(attention_final_layer5))
print("Layer 12 Cosine Similarity 95th:\n", cosine_thresh)
cosine_thresh2 = np.dot(exam_attrib5, attention_all_layer5) / (norm(exam_attrib5)*norm(attention_all_layer5))
print("All layer Cosine Similarity 95th:\n", cosine_thresh2)

Layer 12 Cosine Similarity 95th:
 0.2442624462262068
All layer Cosine Similarity 95th:
 0.13478446227238386


In [820]:
num = np.ceil(2048 * 0.95)
print(2048 - num)
exam_attrib_rank2 = np.copy(ranks_attrib)
exam_attrib_rank2[exam_attrib_rank2 > num] = 0

attention_final_layer_rank2 = np.copy(ranks)
attention_final_layer_rank2[attention_final_layer_rank2 > num] = 0

attention_all_layer_rank2 = np.copy(ranks2)
attention_all_layer_rank2[attention_all_layer_rank2 > num] = 0

102.0


In [821]:
cosine_rank_top = np.dot(exam_attrib_rank2, attention_final_layer_rank2) / (norm(exam_attrib_rank2)*norm(attention_final_layer_rank2))
print("Layer 12 Cosine Similarity 95th ranks:\n", cosine_rank_top)
cosine_rank_top2 = np.dot(exam_attrib_rank2, attention_all_layer_rank2) / (norm(exam_attrib_rank2)*norm(attention_all_layer_rank2))
print("All layer Cosine Similarity 95th ranks:\n", cosine_rank_top2)

Layer 12 Cosine Similarity 95th ranks:
 0.7311890513583853
All layer Cosine Similarity 95th ranks:
 0.7120464365261969


In [822]:
attention_final_layer_top = np.flatnonzero(attention_final_layer5)
attention_final_layer_top = set(attention_final_layer_top)

attention_all_layer_top = np.flatnonzero(attention_all_layer5)
attention_all_layer_top = set(attention_all_layer_top)

exam_attrib_top = np.flatnonzero(exam_attrib5)
exam_attrib_top = set(exam_attrib_top)
print(exam_attrib_top)

{1024, 1025, 514, 1026, 516, 1027, 518, 1536, 1538, 522, 16, 1040, 18, 531, 1552, 2039, 534, 535, 536, 26, 27, 1052, 542, 543, 1060, 551, 555, 565, 569, 583, 1617, 1622, 90, 1628, 612, 613, 102, 103, 104, 1637, 1649, 628, 632, 633, 1157, 1158, 647, 137, 138, 1737, 202, 1738, 1248, 230, 1256, 749, 750, 755, 756, 1793, 261, 266, 267, 781, 276, 279, 816, 820, 825, 834, 862, 863, 1888, 360, 877, 878, 368, 387, 402, 1940, 419, 949, 445, 446, 459, 464, 468, 986, 988, 989, 480, 488, 489, 493, 494, 495, 498, 501, 502, 503, 2040, 1019, 2046}


Grab the tokens stored in the all tokens dictionary so we can know which tokens we are working with as we currently only have the indices.

In [823]:
exam_tokens = all_tokens[str(example)]

Find out which tokens have the highest attentions but not the highest attributions

In [824]:
diff = sorted(list(attention_final_layer_top - exam_attrib_top))
print(len(diff))
diff_tokens = [exam_tokens[idx] for idx in diff]
d_diff = {"token": diff_tokens, "position":diff, "attention_norm":attention_final_layer5[diff], "attribution_norm":exam_attrib5[diff]}
df_diff = pd.DataFrame(d_diff)
df_diff

74


,token,position,attention_norm,attribution_norm
0,.,139,0.798495,0.0
1,talk,171,0.092173,0.0
2,Ġdynamic,189,0.069863,0.0
3,Ġlanguages,190,0.083314,0.0
4,Ġstatic,193,0.057866,0.0
...,...,...,...,...
69,.,1758,0.987677,0.0
70,ĠEvolution,1764,0.071890,0.0
71,.,1869,0.922332,0.0
72,.,1916,0.846734,0.0


In [825]:
print(df_diff['token'].value_counts())

.                  28
Ġlanguages          4
Ġlanguage           3
Ġdynamic            2
Ġassembly           2
Ġoperational        1
Ġcompile            1
ya                  1
ae                  1
een                 1
all                 1
class               1
Ġmigration          1
language            1
Ġprogram            1
Ġx                  1
86                  1
Ġcompilation        1
Ġspecialization     1
Ġcode               1
ilers               1
Ġprox               1
time                1
talk                1
Ġstatic             1
Ġtyped              1
Ġcommitted          1
Ġinteroper          1
Ġtracing            1
IT                  1
iggs                1
Ġvirtual            1
VM                  1
ĠJavaScript         1
Ġnom                1
Ġobject             1
oriented            1
Ġunexpl             1
Ġbenchmarks         1
ĠEvolution          1
Name: token, dtype: int64


Find out which tokens have the highest attributions but not the highest attentions

In [826]:
diff2 = sorted(list(exam_attrib_top - attention_final_layer_top))
print(len(diff))
diff_tokens2 = [exam_tokens[idx] for idx in diff2]
d_diff2 = {"token": diff_tokens2, "position":diff2, "attention_norm": attention_final_layer5[diff2], "attribution_norm":exam_attrib5[diff2]}
df_diff2 = pd.DataFrame(d_diff2)
df_diff2

74


,token,position,attention_norm,attribution_norm
0,ae,16,0.0,0.080537
1,een,18,0.0,0.078617
2,.,26,0.0,0.820988
3,ĠSie,27,0.0,0.081687
4,Ġimplementations,90,0.0,0.089657
...,...,...,...,...
69,ĠHowever,1738,0.0,0.075582
70,Ġtyping,1793,0.0,0.083661
71,",",2039,0.0,0.119385
72,Ġis,2040,0.0,0.135040


In [827]:
print(df_diff2['token'].value_counts())

Ġtyped              17
Ġcode                8
Ġtyping              7
.                    4
Ġlanguage            3
Ġruntime             2
Ġimplementations     2
Ġimplementation      2
,                    2
Ġ(                   2
ability              1
Ġprogrammers         1
and                  1
ae                   1
Ġinteroper           1
Ġwith                1
Ġprogrammer          1
Ġother               1
Ġthe                 1
ĠHowever             1
Ġis                  1
Ġprovide             1
downs                1
ift                  1
36                   1
Ġa                   1
een                  1
ikawa                1
Ġhave                1
Ġlanguages           1
Ġ[                   1
Ġexample             1
ĠFor                 1
ĠSie                 1
Ġan                  1
Name: token, dtype: int64


Find out which tokens are part of the highest attentions and highest attributions.

In [828]:
same = sorted(list(attention_final_layer_top & exam_attrib_top))
print(len(same))
same_tokens = [exam_tokens[idx] for idx in same]
d_same = {"token": same_tokens, "position":same, "attention_norm": attention_final_layer5[same], "attribution_norm":exam_attrib5[same]}
df_same = pd.DataFrame(d_same)
df_same

29


,token,position,attention_norm,attribution_norm
0,.,102,0.741577,0.982889
1,Ġprogram,138,0.054548,0.093605
2,.,202,0.892415,0.233809
3,.,230,0.933252,0.203095
4,Ġcompiler,276,0.080562,0.232910
5,Ġmachine,279,0.059030,0.111264
6,Ġlanguages,360,0.076261,0.122105
7,.,368,0.681094,0.093813
8,Ġlanguages,387,0.068842,0.132378
9,Ġcompiler,402,0.054535,0.191650


In [829]:
print(df_same['token'].value_counts())

.             12
Ġcompiler      5
Ġprograms      3
Ġprogram       2
Ġlanguages     2
Ġcode          2
Ġmachine       1
Ġlanguage      1
Ġsoftware      1
Name: token, dtype: int64
